In [ ]:
## Fetch HTML from Firestore

In [1]:
import os
from dotenv import load_dotenv
from google.cloud import firestore
from google.oauth2 import service_account

# Load environment variables from .env file
load_dotenv()

def get_firestore_client():
    """
    Initialize a Firestore client using a service account file path
    stored in .env as FIREBASE_SERVICE_ACCOUNT_JSON.
    """
    service_account_path = os.getenv("FIREBASE_SERVICE_ACCOUNT_JSON")
    if not service_account_path or not os.path.exists(service_account_path):
        raise FileNotFoundError(
            f"Service account file not found. Check FIREBASE_SERVICE_ACCOUNT_JSON in .env: {service_account_path}"
        )

    credentials = service_account.Credentials.from_service_account_file(service_account_path)
    db = firestore.Client(credentials=credentials, project=credentials.project_id)
    return db


import os
from google.cloud import storage
from google.oauth2 import service_account
from dotenv import load_dotenv

load_dotenv()  # ensure .env is loaded for FIREBASE_SERVICE_ACCOUNT_JSON


def download_text_from_storage(ticker, service_account_path=None):
    """
    Download cleaned 10-K text content from Firebase Storage for a given ticker.
    Path: company_details/EDGAR (US)/filings/{ticker}_10K.txt
    """

    # Get service account path from .env if not provided
    if service_account_path is None:
        service_account_path = os.getenv("FIREBASE_SERVICE_ACCOUNT_JSON")

    if not service_account_path or not os.path.exists(service_account_path):
        raise FileNotFoundError(f"Service account file not found at {service_account_path}")

    # Initialize Firebase Storage client
    credentials = service_account.Credentials.from_service_account_file(service_account_path)
    client = storage.Client(credentials=credentials, project=credentials.project_id)

    bucket_name = "funwai-resume.firebasestorage.app"
    file_path = f"company_details/EDGAR (US)/filings/{ticker}_10K.txt"

    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)

    # Check if the text file exists
    if not blob.exists():
        raise FileNotFoundError(f"File not found for ticker '{ticker}' at path: {file_path}")

    # Download the text content
    text_content = blob.download_as_text(encoding="utf-8")

    return text_content



In [3]:
# test that we are able to pull data from firestore + firebase storage
if __name__ == "__main__":
    print("🔍 Testing Firebase Storage connection...")

    try:
        # Check environment variable
        print("Service account path:", os.getenv("FIREBASE_SERVICE_ACCOUNT_JSON"))

        # Try to download a known file (update ticker if needed)
        ticker = "AAPL"
        html_content = download_text_from_storage(ticker)
        
        # Print preview
        print(f"✅ Successfully downloaded 10k (as text) for {ticker}")
        print("First 500 characters:\n", html_content[:100])

    except Exception as e:
        print("❌ Error:", e)


🔍 Testing Firebase Storage connection...
Service account path: C:/Users/hongn/idealy_new/idealy/backend/funwai-resume-firebase-adminsdk-fbsvc-a956eb6362.json
❌ Error: File not found for ticker 'AAPL' at path: company_details/EDGAR (US)/filings/AAPL_10K.txt


In [11]:
import os
from dotenv import load_dotenv

from openai import OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Create a new OpenAI client with your project key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Select an embeddings model:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## Select an LLM model
llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)


In [7]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What's the weather like in London?"}]
)

print(response.choices[0].message.content)


I'm sorry, but I can't provide real-time weather updates. You can check the current weather in London using a reliable weather website or app.


In [27]:
import pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

# Pinecone setup
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

print(index_name)

pc = Pinecone(api_key=pinecone_api_key)
print(pc.list_indexes())
index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index)


10k-text-rag
[{
    "name": "10k-text-rag",
    "metric": "cosine",
    "host": "10k-text-rag-fqh5rav.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 3072,
    "deletion_protection": "disabled",
    "tags": null
}]


In [25]:
## Create index if it does not exist already
import os
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

# load env
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENV")
index_name = os.getenv("PINECONE_INDEX_NAME")
load_dotenv()

pc = Pinecone(api_key=pinecone_api_key, environment=pinecone_env)

# Delete old index if exists
if index_name in pc.list_indexes():
    pc.delete_index(index_name)

index_name = "10k-text-rag"

if not pc.has_index(index_name):
    pc.create_index(
        name="10k-text-rag",
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [11]:
## Convert HTML to text file and store
import os

ticker = "ABBV"  # or pass dynamically
text_content = download_text_from_storage(ticker)

output_folder = "C:/Users/hongn/idealy_new/idealy/backend/rag-api/clean_10k_texts"

# Save cleaned text
output_path = os.path.join(output_folder, f"{ticker}_10K.txt")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(text_content)

print(f"✅ Saved cleaned text to: {output_path}")

FileNotFoundError: File not found for ticker 'ABBV' at path: company_details/EDGAR (US)/filings/ABBV_10K.txt

In [29]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Path to the folder containing your cleaned text files
TEXT_FOLDER = "./clean_10k_texts"

# Load all .txt files
loader = DirectoryLoader(
    TEXT_FOLDER,
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

docs = loader.load()

print(f"✅ Loaded {len(docs)} documents.\n")

# Preview first document
print("First document metadata:", docs[0].metadata)
print("\n--- First 500 characters ---\n")
print(docs[0].page_content[:500])


✅ Loaded 124 documents.

First document metadata: {'source': 'clean_10k_texts\\AAPL_2025_10K.txt'}

--- First 500 characters ---

| |   
---|---|---  

UNITED STATES

SECURITIES AND EXCHANGE COMMISSION

Washington, D.C. 20549

| |   
---|---|---  

FORM 10-K

| |   
---|---|---  

(Mark One)

☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the fiscal year ended September 27, 2025

or

☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the transition period from  to  .

Commission File Number: 001-36743

| |   
---|---|---  

Apple Inc.


In [29]:
## Load documents
import os
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

# Path to the folder containing your cleaned text files
TEXT_FOLDER = "./clean_10k_texts"  # e.g., output of clean_10k_html()

# Option 1: Load a single file
single_file_path = os.path.join(TEXT_FOLDER, "NFLX_10K.txt")
loader = TextLoader(single_file_path, encoding="utf-8")
docs = loader.load()

print(f"✅ Loaded 1 document with {len(docs[0].page_content)} characters.")
print(docs[0].page_content[:500])

# Option 2: Load all text files in a folder
# loader = DirectoryLoader(TEXT_FOLDER, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
# docs = loader.load()

# print(f"✅ Loaded {len(docs)} documents.")
# print(docs[0].metadata)
# print(docs[0].page_content[:500])


✅ Loaded 1 document with 279148 characters.
| |   
---|---|---  
UNITED STATES SECURITIES AND EXCHANGE COMMISSION

Washington, D.C. 20549

_____________________________________________________________________

FORM 10-K

_____________________________________________________________________

(Mark One)

| | | | |   
---|---|---|---|---|---  
☒| ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  

For the fiscal year ended December 31, 2024

OR

| | | | |   
---|---|---|---|---|---  
☐| TRANSITION REPORT P


In [31]:
## Docs to Chunks via textsplitter functions
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200, 
    add_start_index=True)

chunks = text_splitter.split_documents(docs)
print(f"✅ Split into {len(chunks)} chunks.")

✅ Split into 93427 chunks.


In [33]:
## load chunks into the Pinecone Vector store ('vector_store' defined earlier)
from tqdm import tqdm
import time

BATCH_SIZE = 200  # You can tune this (100–300 is ideal)
document_ids = []

for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="Uploading chunks"):
    batch = chunks[i:i + BATCH_SIZE]

    # Retry logic (recommended for Pinecone)
    for attempt in range(3):
        try:
            ids = vector_store.add_documents(documents=batch)
            document_ids.extend(ids)
            break  # break retry loop
        except Exception as e:
            print(f"⚠️ Error uploading batch {i//BATCH_SIZE}, attempt {attempt+1}: {e}")
            time.sleep(2)  # wait + retry


Uploading chunks:  67%|█████████████████████████████████████████▌                    | 314/468 [25:49<12:17,  4.79s/it]

⚠️ Error uploading batch 314, attempt 1: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:46:47 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '96', 'x-pinecone-request-id': '7982488119216157449', 'x-envoy-upstream-service-time': '28', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}

⚠️ Error uploading batch 314, attempt 2: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:46:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '66', 'x-pinecone-request-id': '1847917596060236855', 'x-envoy-upstream-service-time': '6', 'server': 'envoy'})
HTTP response body: {"code":8,"messag

Uploading chunks:  67%|█████████████████████████████████████████▋                    | 315/468 [26:07<22:43,  8.91s/it]

⚠️ Error uploading batch 315, attempt 1: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:47:03 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '99', 'x-pinecone-request-id': '6887225196364178439', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}

⚠️ Error uploading batch 315, attempt 2: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:47:09 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '163', 'x-pinecone-request-id': '1152302126835574217', 'x-envoy-upstream-service-time': '5', 'server': 'envoy'})
HTTP response body: {"code":8,"messag

Uploading chunks:  68%|█████████████████████████████████████████▊                    | 316/468 [26:26<29:52, 11.79s/it]

...

⚠️ Error uploading batch 324, attempt 1: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:49:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '128', 'x-pinecone-request-id': '7256124030153667348', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}

⚠️ Error uploading batch 324, attempt 2: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:50:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '70', 'x-pinecone-request-id': '7280234316967944236', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":8,"messag

Uploading chunks:  69%|███████████████████████████████████████████                   | 325/468 [29:20<44:46, 18.79s/it]

⚠️ Error uploading batch 325, attempt 1: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:50:18 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '110', 'x-pinecone-request-id': '7934570794545337204', 'x-envoy-upstream-service-time': '6', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}

⚠️ Error uploading batch 325, attempt 2: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 17 Nov 2025 18:50:24 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '92', 'x-pinecone-request-id': '6497272980675131265', 'x-envoy-upstream-service-time': '5', 'server': 'envoy'})
HTTP response body: {"code":8,"messag

Uploading chunks:  69%|███████████████████████████████████████████                   | 325/468 [29:37<13:02,  5.47s/it]


KeyboardInterrupt: 

In [ ]:
print(len(document_ids))
vector_store.similarity_search("What does Apple say about revenue recognition?", k=2)
# the above returns 2 relevant chunks

In [43]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [45]:
from langchain_openai import ChatOpenAI

# Example using local API key or .env
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

In [47]:
from langchain.agents import create_agent

tools = [retrieve_context]

# System prompt / instructions for the agent
system_prompt = (
    "You are a helpful assistant that answers user queries using the provided context. "
    "Use the retrieval tool when necessary."
    "provide succinct answers and as quantitative answers as available from retrieval"
)

# Create the agent
agent = create_agent(llm, tools=tools, system_prompt=system_prompt)


In [59]:
query = (
    "How does Morgan Stanley make money??\n\n"
    "Once you get the answer, please find some of the products and services they sell and how much money each product made.\n\n"
    "Who are Morgan Stanley's main competitors?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    msg = event["messages"][-1]

    # Only print the final AI messages
    if getattr(msg, "type", None) == "ai": # do not get human, tool messages, only the AI response
        print(msg.content)




Morgan Stanley makes money primarily through the following business segments:

1. Investment Banking: Revenue from financial advisory and underwriting assignments.
2. Investment Management: Revenue from providing asset management and wealth advisory services.
3. Commissions and Fees: Revenue from executing and clearing client transactions on stock, options, and futures exchanges, as well as over-the-counter (OTC) transactions.
4. Markets: Revenue from trading activities, including equity and fixed income markets.

Products and Services & Revenue (latest available):
- Markets (trading, including equities and fixed income): $19.8 billion in revenue.
  - Equity Markets: Growth driven by cash equities, equity derivatives, and prime services.
  - Fixed Income Markets: Growth in asset-backed financing, securitization, and underwriting fees.
- Investment Banking and Investment Management: Specific revenue figures for these segments are not provided in the retrieved context, but they are maj